In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np
import keras
from keras import layers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

--2021-02-10 17:15:25--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.31.128, 172.217.204.128, 173.194.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2021-02-10 17:15:26 (65.8 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [3]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [7]:

model = keras.models.Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(layers.Bidirectional(layers.LSTM(128)))
model.add(layers.Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')



In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               234496    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              825227    
Total params: 1,380,823
Trainable params: 1,380,823
Non-trainable params: 0
_________________________________________________________________


In [9]:
address="./kaggle/working/weights-improvement.hdf5"
stop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.0001, 
                             patience = 5, verbose = 1, mode = 'auto')
save = ModelCheckpoint(address, monitor = 'val_accuracy', 
                               verbose = 0, save_best_only = True)
callbacks = [stop, save]

history = model.fit(predictors, label, validation_split=0.20, epochs=50, verbose=1, callbacks = callbacks)

Epoch 1/50
387/387 [==============================] - 5s 12ms/step - loss: 6.9160 - accuracy: 0.0228 - val_loss: 6.8253 - val_accuracy: 0.0162
Epoch 2/50
387/387 [==============================] - 4s 10ms/step - loss: 6.4356 - accuracy: 0.0310 - val_loss: 6.8735 - val_accuracy: 0.0223
Epoch 3/50
387/387 [==============================] - 4s 10ms/step - loss: 6.2292 - accuracy: 0.0371 - val_loss: 6.9775 - val_accuracy: 0.0394
Epoch 4/50
387/387 [==============================] - 4s 10ms/step - loss: 5.9742 - accuracy: 0.0473 - val_loss: 7.0733 - val_accuracy: 0.0378
Epoch 5/50
387/387 [==============================] - 4s 11ms/step - loss: 5.6750 - accuracy: 0.0595 - val_loss: 7.2186 - val_accuracy: 0.0440
Epoch 6/50
387/387 [==============================] - 4s 12ms/step - loss: 5.3371 - accuracy: 0.0741 - val_loss: 7.4069 - val_accuracy: 0.0414
Epoch 7/50
387/387 [==============================] - 4s 10ms/step - loss: 4.9899 - accuracy: 0.0889 - val_loss: 7.6088 - val_accuracy: 0.0482

Tried with val_accuracy, but still the training stopped as it did not see the improvement mentioned via min_delta = 0.0001.